In [352]:
# Required Imports
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re as re

We first get the company page that the user wants to scrape. We then check for an existing user credentials file or create one if its a first time user

In [353]:
#Get the Company's Linkedin URL from User
page = input("Enter the Company Linkedin URL: ")
company_name = page[33:-1]

#See if existing user credential file exists or create one 
try:
    f= open("linkedin_credentials.txt","r")
    contents = f.read()
    username = contents.replace("=",",").split(",")[1]
    password = contents.replace("=",",").split(",")[3]
except:
    f= open("linkedin_credentials.txt","w+")
    username = input('Enter your linkedin username: ')
    password = input('Enter your linkedin password: ')
    f.write("username={}, password={}".format(username,password))
    f.close()

Enter the Company Linkedin URL: https://www.linkedin.com/school/udacity/


We now need to get Selenium to use ChromeDriver to open chrome and visit Linkedin where it will sign in using your login info

In [ ]:
#accessing Chromedriver
browser = webdriver.Chrome('chromedriver')


#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)
elementID.submit()

## Scrolling Down the Page
We next search for the Linkedin page that we want to scrape. I’m into backpacking and the outdoors so I’ll use REI as an example. Now, this next section of code allows us to scroll down the entire Linkedin page. This is important because with an infinite scroll page such as Linkedin you would only get the first few posts without this step.

In [ ]:
#Go to company post webpage
browser.get(page + 'posts/')


#Simulate scrolling to capture all posts
SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

## Enter Beautiful Soup…
We will now collect the source code and then starting looking for the page elements that we want to access.

In [ ]:
company_page = browser.page_source

#Check out page source code
linkedin_soup = bs(company_page.encode("utf-8"), "html")
linkedin_soup.prettify()

containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})

## Finding the elements that we want

In [ ]:
post_dates = []
post_texts = []
post_likes = []
post_comments = []
post_shared = []
video_views = []
media_links = []
media_type = []

for container in containers:

    try:
        posted_date = container.find("span",{"class":"visually-hidden"})
        text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper"})
        text = text_box.find("span",{"dir":"ltr"})
        post_shares = container.findAll("li", {'class':["social-details-social-counts__item--with-social-proof","social-details-social-counts__comments social-details-social-counts__item"]})
        new_likes = container.findAll("li", {'class':["social-details-social-counts__item--with-social-proof","social-details-social-counts__item"]})
        new_comments = container.findAll("li", {"class":["social-details-social-counts__item" "social-details-social-counts__comments social-details-social-counts__item--with-social-proof"]})

#         new_comments = container.findAll("button", {"class": "t-black--light t-12 hoverable-link-text"})
        
        post_dates.append(posted_date.text.strip())
        post_texts.append(text.text.strip())
        





        try:
            video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
            video_link = video_box[0].find("video", {"class":"vjs-tech"})
            media_links.append(video_link['src'])
            media_type.append("Video")
        except:
            try:
                image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
                media_links.append(image_link['src'])
                media_type.append("Image")
            except:
                try:
                    #mutiple shared images
                    image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                    image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
                    media_links.append(image_link['src'])
                    media_type.append("Multiple Images")
                except:
                    try:
                        article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
                        article_link = article_box[0].find('a', href=True)
                        media_links.append(article_link['href'])
                        media_type.append("Article")
                    except:
                        try:
                            video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
                            video_link = video_box[0].find('a', href=True)
                            media_links.append(video_link['href'])
                            media_type.append("Youtube Video")   
                        except:
                            try:
                                poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                                media_links.append("None")
                                media_type.append("Other: Poll, Shared Post, etc")
                            except:
                                media_links.append("None")
                                media_type.append("Unknown")



        #Getting Video Views. (The folling three lines prevents class name overlap)
#         view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
#         view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
#         result = view_container2 - view_container1
#         view_container = []
#         for i in result:
#             view_container += i

#         try:
#             video_views.append(view_container[1].text.strip().replace(' Views',''))

#         except:
#             video_views.append('N/A')


        try:
            post_likes.append(new_likes[0].text.strip())
        except:
            post_likes.append(0)
            pass

        try:
            post_comments.append(new_comments[0].text.strip())                           
        except:                                                           
            post_comments.append(0)
            pass
        
        try:
            post_shared.append(post_shares[1].text.strip())                           
        except:                                                           
            post_shared.append(0)
            pass
    
    except:
        pass

In [ ]:
#Stripping non-numeric values
comment_count = []
for i in post_comments:
    s = str(i).replace('comments','').replace('shares','').replace(' ','')
    comment_count += [s]

In [ ]:
print(post_comments)

In [ ]:
print(post_shared)

## Organizing the Data for Export
Finally, we are ready to organize our collected data into a Pandas DataFrame and then export it as a CSV or Excel file. You can find this file saved in the same folder that you ran the program from

In [ ]:
data = {
    "Date_Posted": post_dates,
    "Media_Type": media_type,
    "Post_Caption": post_texts,
    "Post_Likes_Counts": post_likes,
    "Post_Comments_Counts": post_comments,
    "Post_Shared_Counts" : post_shared,
#     "Video Views": video_views,
#     "Media Links": media_links
}

df = pd.DataFrame(data)
df

In [ ]:
# #Exporting as csv file to program folder
# df.to_csv("{}.csv".format(company_name), encoding='utf-8', index=False)

In [ ]:
df.shape

In [ ]:
#Export to Excel file to program folder
writer = pd.Excelwriter = pd.ExcelWriter("{}.xlsx".format(company_name), engine='xlsxwriter')
df.to_excel(writer, index =False)
writer.save()